In [1]:
import requests

def call_large_model_api(user_prompt, user_input):
    url = 'http://117.161.233.106:8000/v1/chat/completions?model=llama'  # 请替换为实际的API URL
    headers = {
        'Content-Type': 'application/json',
        # 请替换为实际的API密钥
    }
    data = {
        "model": "llama3",
        "stream": False,
        "temperature": 0.01,
        "max_tokens": 1024,
        "repetition_penalty": 10,
        "top_p": 0.8,
        "do_sample": True,
        "messages": [
            {
                "role": "system",
                "content": user_prompt
            },
            {
                "role": "user",
                "content": user_input
            }
        ]
    }
    
    response = requests.post(url, json=data, headers=headers)
    
    return response


In [2]:
import concurrent.futures
import csv
import requests
from tqdm import tqdm



# 处理每一行的函数
def process_row(row):
    # 处理行的逻辑，例如提取入职要求和工作内容
    data = row[18]  # 假定这是包含工作描述的列
    try:
        # 调用大模型API提取技术栈
        prompt = "从任职要求提取技术栈\n- Return format: 技术栈: \n1. \n2...."
        response = call_large_model_api(prompt, data)
        skill = response.json()
        skills = skill["choices"][0]["message"]["content"]
    except Exception as e:
        print(f"Error processing row: {e}")
        skills = ""
    
    row.append(skills)
    return row

# 主函数
def main():
    # 打开输入和输出文件
    with open('Boss直聘--修_1.csv', 'r', newline='', encoding='utf-8') as csvfile, \
         open('output_file.csv', 'w', newline='', encoding='utf-8') as output_file:

        reader = csv.reader(csvfile)
        writer = csv.writer(output_file)
        
        # 写入标题行
        writer.writerow(['职位名称', '工作地址','学历要求', '工作年限要求','招聘人数','薪资待遇','公司行业','公司性质','公司规模','融资阶段','招聘状态','职位类型','岗位描述','公司介绍','公司工商信息','简历详情页地址','更新日期','工作内容','入职要求', '技术栈'])
        next(reader)  # 跳过标题行
        
        # 初始化一个列表来存储所有的行，以便并行处理
        rows = [row for row in reader]
        
        # 使用线程池并行处理数据
        with concurrent.futures.ThreadPoolExecutor(max_workers=30) as executor:
            # 提交所有任务并获取future对象列表
            futures = [executor.submit(process_row, row) for row in rows]
            
            # 初始化计数器和批处理列表
            count = 0
            batch = []
            
            # 遍历future对象，获取结果并写入文件
            for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="Processing rows", unit="row"):
                # 结果处理
                processed_row = future.result()
                batch.append(processed_row)
                count += 1
                
                # 每100行，写入文件并清空批处理列表
                if count % 100 == 0:
                    writer.writerows(batch)
                    batch.clear()

# 运行主函数
if __name__ == "__main__":
    main()
# 50 60 54
# 30 60 66
# 20 60 63

# 10 60 62
# 6  60 50


Processing rows:   7%|▋         | 67/1000 [01:11<11:18,  1.37row/s] 